In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset5') 

# Actions that we try to detect
actions = np.array(['Address', 'Home', 'time'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'Address': 0, 'Home': 1, 'time': 2}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(90, 30, 1662)

In [13]:
np.array(labels).shape

(90,)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(90, 30, 1662)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
y_test.shape

(5, 3)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 14s 2s/step - loss: 1.2134 - categorical_accuracy: 0.3529
Epoch 2/2000
3/3 [==============================] - 1s 226ms/step - loss: 8.9530 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 135ms/step - loss: 8.0342 - categorical_accuracy: 0.3176
Epoch 4/2000
3/3 [==============================] - 0s 155ms/step - loss: 6.8175 - categorical_accuracy: 0.3176
Epoch 5/2000
3/3 [==============================] - 0s 143ms/step - loss: 6.6564 - categorical_accuracy: 0.4824
Epoch 6/2000
3/3 [==============================] - 0s 118ms/step - loss: 6.3800 - categorical_accuracy: 0.3176
Epoch 7/2000
3/3 [==============================] - 1s 388ms/step - loss: 2.8415 - categorical_accuracy: 0.4824
Epoch 8/2000
3/3 [==============================] - 0s 161ms/step - loss: 19.0336 - categorical_accuracy: 0.3294
Epoch 9/2000
3/3 [==============================] - 0s 135ms/step - loss: 9.8826 - categorical_accuracy: 

3/3 [==============================] - 1s 364ms/step - loss: 1.8219 - categorical_accuracy: 0.3529
Epoch 74/2000
3/3 [==============================] - 1s 213ms/step - loss: 2.0779 - categorical_accuracy: 0.4235
Epoch 75/2000
3/3 [==============================] - 0s 150ms/step - loss: 2.3987 - categorical_accuracy: 0.4353
Epoch 76/2000
3/3 [==============================] - 0s 161ms/step - loss: 1.7919 - categorical_accuracy: 0.3765
Epoch 77/2000
3/3 [==============================] - 0s 159ms/step - loss: 1.4424 - categorical_accuracy: 0.4235
Epoch 78/2000
3/3 [==============================] - 1s 295ms/step - loss: 1.9994 - categorical_accuracy: 0.3882
Epoch 79/2000
3/3 [==============================] - 1s 282ms/step - loss: 1.6718 - categorical_accuracy: 0.3882
Epoch 80/2000
3/3 [==============================] - 0s 167ms/step - loss: 1.6794 - categorical_accuracy: 0.4353
Epoch 81/2000
3/3 [==============================] - 0s 162ms/step - loss: 1.5654 - categorical_accuracy: 0.41

3/3 [==============================] - 0s 129ms/step - loss: 0.2611 - categorical_accuracy: 0.8941
Epoch 145/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.2652 - categorical_accuracy: 0.9294
Epoch 146/2000
3/3 [==============================] - 0s 175ms/step - loss: 0.2394 - categorical_accuracy: 0.9294
Epoch 147/2000
3/3 [==============================] - 0s 123ms/step - loss: 0.2583 - categorical_accuracy: 0.9059
Epoch 148/2000
3/3 [==============================] - 0s 142ms/step - loss: 0.2555 - categorical_accuracy: 0.8706
Epoch 149/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.2567 - categorical_accuracy: 0.8471
Epoch 150/2000
3/3 [==============================] - 0s 150ms/step - loss: 0.2175 - categorical_accuracy: 0.9529
Epoch 151/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.2031 - categorical_accuracy: 0.8471
Epoch 152/2000
3/3 [==============================] - 0s 119ms/step - loss: 0.1945 - categorical_accura

3/3 [==============================] - 1s 122ms/step - loss: 0.0510 - categorical_accuracy: 0.9765
Epoch 216/2000
3/3 [==============================] - 0s 140ms/step - loss: 0.0573 - categorical_accuracy: 0.9765
Epoch 217/2000
3/3 [==============================] - 1s 375ms/step - loss: 0.0560 - categorical_accuracy: 0.9765
Epoch 218/2000
3/3 [==============================] - 1s 293ms/step - loss: 0.1088 - categorical_accuracy: 0.9765
Epoch 219/2000
3/3 [==============================] - 0s 162ms/step - loss: 0.1482 - categorical_accuracy: 0.9294
Epoch 220/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.1678 - categorical_accuracy: 0.9647
Epoch 221/2000
3/3 [==============================] - 1s 353ms/step - loss: 0.1518 - categorical_accuracy: 0.9647
Epoch 222/2000
3/3 [==============================] - 1s 218ms/step - loss: 0.1135 - categorical_accuracy: 0.9765
Epoch 223/2000
3/3 [==============================] - 0s 126ms/step - loss: 0.1342 - categorical_accura

3/3 [==============================] - 0s 131ms/step - loss: 0.0036 - categorical_accuracy: 1.0000
Epoch 287/2000
3/3 [==============================] - 0s 143ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 0s 151ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 289/2000
3/3 [==============================] - 0s 157ms/step - loss: 0.0034 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 139ms/step - loss: 0.0034 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 0s 149ms/step - loss: 0.0034 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 174ms/step - loss: 0.0033 - categorical_accuracy: 1.0000
Epoch 293/2000
3/3 [==============================] - 0s 137ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 294/2000
3/3 [==============================] - 0s 132ms/step - loss: 0.0033 - categorical_accura

...

3/3 [==============================] - 0s 68ms/step - loss: 5.2626e-05 - categorical_accuracy: 1.0000
Epoch 1662/2000
3/3 [==============================] - 0s 62ms/step - loss: 5.2395e-05 - categorical_accuracy: 1.0000
Epoch 1663/2000
3/3 [==============================] - 0s 81ms/step - loss: 5.2308e-05 - categorical_accuracy: 1.0000
Epoch 1664/2000
3/3 [==============================] - 1s 264ms/step - loss: 5.2172e-05 - categorical_accuracy: 1.0000
Epoch 1665/2000
3/3 [==============================] - 1s 307ms/step - loss: 5.2119e-05 - categorical_accuracy: 1.0000
Epoch 1666/2000
3/3 [==============================] - 0s 51ms/step - loss: 5.2252e-05 - categorical_accuracy: 1.0000
Epoch 1667/2000
3/3 [==============================] - 0s 153ms/step - loss: 5.1847e-05 - categorical_accuracy: 1.0000
Epoch 1668/2000
3/3 [==============================] - 0s 109ms/step - loss: 5.1673e-05 - categorical_accuracy: 1.0000
Epoch 1669/2000
3/3 [==============================] - 0s 138ms/step

3/3 [==============================] - 0s 100ms/step - loss: 4.5712e-05 - categorical_accuracy: 1.0000
Epoch 1730/2000
3/3 [==============================] - 0s 80ms/step - loss: 4.5667e-05 - categorical_accuracy: 1.0000
Epoch 1731/2000
3/3 [==============================] - 0s 79ms/step - loss: 4.5400e-05 - categorical_accuracy: 1.0000
Epoch 1732/2000
3/3 [==============================] - 0s 98ms/step - loss: 4.5519e-05 - categorical_accuracy: 1.0000
Epoch 1733/2000
3/3 [==============================] - 0s 65ms/step - loss: 4.5248e-05 - categorical_accuracy: 1.0000
Epoch 1734/2000
3/3 [==============================] - 0s 77ms/step - loss: 4.5344e-05 - categorical_accuracy: 1.0000
Epoch 1735/2000
3/3 [==============================] - 0s 78ms/step - loss: 4.5143e-05 - categorical_accuracy: 1.0000
Epoch 1736/2000
3/3 [==============================] - 0s 80ms/step - loss: 4.4968e-05 - categorical_accuracy: 1.0000
Epoch 1737/2000
3/3 [==============================] - 0s 74ms/step - l

3/3 [==============================] - 0s 58ms/step - loss: 3.9798e-05 - categorical_accuracy: 1.0000
Epoch 1799/2000
3/3 [==============================] - 0s 66ms/step - loss: 3.9616e-05 - categorical_accuracy: 1.0000
Epoch 1800/2000
3/3 [==============================] - 0s 83ms/step - loss: 3.9717e-05 - categorical_accuracy: 1.0000
Epoch 1801/2000
3/3 [==============================] - 0s 67ms/step - loss: 3.9257e-05 - categorical_accuracy: 1.0000
Epoch 1802/2000
3/3 [==============================] - 0s 60ms/step - loss: 3.9323e-05 - categorical_accuracy: 1.0000
Epoch 1803/2000
3/3 [==============================] - 0s 77ms/step - loss: 3.9438e-05 - categorical_accuracy: 1.0000
Epoch 1804/2000
3/3 [==============================] - 0s 72ms/step - loss: 3.9209e-05 - categorical_accuracy: 1.0000
Epoch 1805/2000
3/3 [==============================] - 0s 94ms/step - loss: 3.9073e-05 - categorical_accuracy: 1.0000
Epoch 1806/2000
3/3 [==============================] - 0s 66ms/step - lo

3/3 [==============================] - 0s 78ms/step - loss: 3.4670e-05 - categorical_accuracy: 1.0000
Epoch 1867/2000
3/3 [==============================] - 0s 83ms/step - loss: 3.4548e-05 - categorical_accuracy: 1.0000
Epoch 1868/2000
3/3 [==============================] - 0s 76ms/step - loss: 3.4463e-05 - categorical_accuracy: 1.0000
Epoch 1869/2000
3/3 [==============================] - 0s 112ms/step - loss: 3.4458e-05 - categorical_accuracy: 1.0000
Epoch 1870/2000
3/3 [==============================] - 0s 162ms/step - loss: 3.4292e-05 - categorical_accuracy: 1.0000
Epoch 1871/2000
3/3 [==============================] - 0s 157ms/step - loss: 3.4182e-05 - categorical_accuracy: 1.0000
Epoch 1872/2000
3/3 [==============================] - 0s 142ms/step - loss: 3.4151e-05 - categorical_accuracy: 1.0000
Epoch 1873/2000
3/3 [==============================] - 0s 101ms/step - loss: 3.4212e-05 - categorical_accuracy: 1.0000
Epoch 1874/2000
3/3 [==============================] - 0s 115ms/ste

3/3 [==============================] - 0s 117ms/step - loss: 3.0470e-05 - categorical_accuracy: 1.0000
Epoch 1935/2000
3/3 [==============================] - 0s 139ms/step - loss: 3.0305e-05 - categorical_accuracy: 1.0000
Epoch 1936/2000
3/3 [==============================] - 0s 149ms/step - loss: 3.0263e-05 - categorical_accuracy: 1.0000
Epoch 1937/2000
3/3 [==============================] - 0s 71ms/step - loss: 3.0205e-05 - categorical_accuracy: 1.0000
Epoch 1938/2000
3/3 [==============================] - 0s 84ms/step - loss: 3.0114e-05 - categorical_accuracy: 1.0000
Epoch 1939/2000
3/3 [==============================] - 0s 129ms/step - loss: 3.0002e-05 - categorical_accuracy: 1.0000
Epoch 1940/2000
3/3 [==============================] - 0s 104ms/step - loss: 2.9919e-05 - categorical_accuracy: 1.0000
Epoch 1941/2000
3/3 [==============================] - 1s 406ms/step - loss: 2.9890e-05 - categorical_accuracy: 1.0000
Epoch 1942/2000
3/3 [==============================] - 0s 145ms/st

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [25]:
res = model.predict(X_test)

In [26]:
actions[np.argmax(res[4])]

'Home'

In [27]:
actions[np.argmax(y_test[4])]

'Home'

In [28]:
model.save('mod5.h5')

In [29]:
model.load_weights('mod5.h5')

In [30]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [31]:

yhat = model.predict(X_test)

In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[4, 0],
        [0, 1]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [34]:
accuracy_score(ytrue, yhat)

1.0

In [35]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<cl

time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<cl

time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
time
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<cl

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Home
<class '

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Address
